## Setup Pytorch
First, import the Pytorch dependencies.

Then we define the remote workers Alice and Bob, who will host the remote data while a local worker (or client) will manage the learning task. Note that we use virtual workers: these workers act just like normal remote workers except that they exist within the same Python program. Thus, we still serialize the commands to be exchanged between the workers but we don’t actually send them over the network. This approach allows us to avoid network-related issues and focus on evaluation of the approach on a local setup


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as functional
import torch.optim as optim
from torchvision import datasets, transforms
import syft

In [2]:
hook = syft.TorchHook(torch)  # Add extra functionalities to PyTorch for Federated Learning
bob_worker = syft.VirtualWorker(hook, id="bob")  # Define remote worker Bob
alice_worker = syft.VirtualWorker(hook, id="alice")

# Define Learning Task Settings

class TrainingArgs():
    def __init__(self):
        self.batch_size = 100
        self.test_batch_size = 1000
        self.epochs = 10  # Default value for epochs
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 30
        self.save_model = False

args = TrainingArgs()
use_cuda = not args.no_cuda and torch.cuda.is_available()  # Use GPU if available unless CPU is specified
torch.manual_seed(args.seed)
device = torch.device("cuda" if use_cuda else "cpu")
data_loader_kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

## Load Data and Send to Workers
In this scenario, the data is loaded and transformed from a local training dataset into a federated dataset using the `.federate` method: the dataset is split into two parts and sent to the workers Alice and Bob. This federated dataset is then given to a federated DataLoader which will iterate over remote batches. The test dataset remains unchanged as the local client will perform the test evaluation.


In [3]:
federated_train_loader = syft.FederatedDataLoader(
    datasets.CIFAR10(
        "../data", train=True, download=True,
        transform=transforms.Compose(
            [transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]
        ),
    ).federate((bob_worker, alice_worker)),  # Distribute the dataset across all workers
    batch_size=args.batch_size,
    shuffle=True,
    **data_loader_kwargs
)

test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(
        "../data", train=False,
        transform=transforms.Compose(
            [transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]
        ),
    ),
    batch_size=args.test_batch_size,
    shuffle=True,
    **data_loader_kwargs
)


Files already downloaded and verified


## Convolutional Neural Network Definition
- Two Convolutional Layers: 20 kernels of size 5x5 and 50 kernels of size 5x5
- Two Fully Connected Layers with 500 and 10 neurons

In [9]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(50 * 5 * 5, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = functional.relu(self.conv1(x))
        x = functional.max_pool2d(x, 2, 2)
        x = functional.relu(self.conv2(x))
        x = functional.max_pool2d(x, 2, 2)
        x = x.view(-1, 50*5*5)
        x = functional.relu(self.fc1(x))
        x = self.fc2(x)
        return functional.log_softmax(x, dim=1)

## Define Training and Testing Functions
For training, since the data batches are distributed across Alice and Bob, the model must be sent to the correct location for each batch. Then, we perform all operations remotely with the same syntax as if we were working in a local PyTorch environment. Once finished, we retrieve the model updates and the loss to monitor improvements using the `.get()` method.


In [5]:
def train_model(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(federated_train_loader):  # Now it is a distributed dataset
        model.send(data.location)  # Send the model to the correct location
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = functional.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        model.get()  # Retrieve the model back
        if batch_idx % args.log_interval == 0:
            loss = loss.get()  # Retrieve the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(train_loader) * args.batch_size,
                100. * batch_idx / len(train_loader), loss.item()))

def test_model(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += functional.nll_loss(output, target, reduction='sum').item()  # Sum up batch loss
            pred = output.argmax(1, keepdim=True)  # Get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

## Start Training

In [10]:
model = SimpleCNN().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr)

for epoch in range(1, args.epochs + 1):
    train_model(args, model, device, federated_train_loader, optimizer, epoch)
    test_model(args, model, device, test_loader)

if args.save_model:
    torch.save(model.state_dict(), "mnist_cnn.pt")

Train Epoch: 1 [0/50000 (0%)]	Loss: 2.300576
Train Epoch: 1 [3000/50000 (6%)]	Loss: 2.291166
Train Epoch: 1 [6000/50000 (12%)]	Loss: 2.276882
Train Epoch: 1 [9000/50000 (18%)]	Loss: 2.219726
Train Epoch: 1 [12000/50000 (24%)]	Loss: 2.206737
Train Epoch: 1 [15000/50000 (30%)]	Loss: 2.124860
Train Epoch: 1 [18000/50000 (36%)]	Loss: 2.037491
Train Epoch: 1 [21000/50000 (42%)]	Loss: 1.972625
Train Epoch: 1 [24000/50000 (48%)]	Loss: 1.956890
Train Epoch: 1 [27000/50000 (54%)]	Loss: 1.941134
Train Epoch: 1 [30000/50000 (60%)]	Loss: 1.781886
Train Epoch: 1 [33000/50000 (66%)]	Loss: 1.881317
Train Epoch: 1 [36000/50000 (72%)]	Loss: 1.930419
Train Epoch: 1 [39000/50000 (78%)]	Loss: 1.764993
Train Epoch: 1 [42000/50000 (84%)]	Loss: 1.713211
Train Epoch: 1 [45000/50000 (90%)]	Loss: 1.708090
Train Epoch: 1 [48000/50000 (96%)]	Loss: 1.824077

Test set: Average loss: 1.7678, Accuracy: 3726/10000 (37%)

Train Epoch: 2 [0/50000 (0%)]	Loss: 1.732595
Train Epoch: 2 [3000/50000 (6%)]	Loss: 1.852025
Train